# Abstraction and technologies

Git uses a number of abstractions, technologies, and tools behind the scenes, organized in the .git folder hierarchy.

* filesystem
* hash function, CHF SHA-1
* compression
* key-value store
* graphs: trees, acyclic directed graphs ( DAGs )


<br />
<br />
<br />
<br />
<br />

## Filesystem

In [1]:
!tree .git

.git
├── branches
├── COMMIT_EDITMSG
├── config
├── description
├── HEAD
├── hooks
│   ├── applypatch-msg.sample
│   ├── commit-msg.sample
│   ├── fsmonitor-watchman.sample
│   ├── post-update.sample
│   ├── pre-applypatch.sample
│   ├── pre-commit.sample
│   ├── pre-merge-commit.sample
│   ├── prepare-commit-msg.sample
│   ├── pre-push.sample
│   ├── pre-rebase.sample
│   ├── pre-receive.sample
│   └── update.sample
├── index
├── info
│   └── exclude
├── logs
│   ├── HEAD
│   └── refs
│       └── heads
│           └── master
├── objects
│   ├── a1
│   │   └── 3eb9d02b9ee2c2f0d073bbc65d91a18c7e7316
│   ├── a5
│   │   └── c19667710254f835085b99726e523457150e03
│   ├── e5
│   │   └── dbedc48e87928e4a8222d070bad6dab173ff30
│   ├── info
│   └── pack
└── refs
    ├── heads
    │   └── master
    └── tags

15 directories, 24 files


A key feature of the file system is that no two files can have the same path/filename.  Those must be unique.

In [2]:
!ls -Fld confusion

drwxr-xr-x 2 jovyan users 4096 Jun  9 18:36 confusion/


In [3]:
!find confusion

confusion
confusion/dup1  
confusion/dup1
confusion/dup1 
confusion/dup1   


In [4]:
!find confusion | sort | cat -vet

confusion$
confusion/dup1$
confusion/dup1 $
confusion/dup1  $
confusion/dup1   $


<br />
<br />
<br />
<br />
<br />

## Hash functions
* Cryptographic hash functions ( CHF )
* SHA-1

### Properties, features
- fast to compute: O(n)
- uniform size: maps data of arbitrary size to a fixed size ( SHA-1: 40 hex digits )
- hex characters
- one-way: given a hash output, infeasible to generate input; irreversible
- deterministic: same input, same output
    - if two hashes differ, their inputs differ
- unique: infeasible that different inputs generate same output; collision
- sensitive to change: small input change, large output change; avalanche effect


In [5]:
%%bash
<<< "hello world" wc -c
<<< "hello world" shasum -a 1
<<< "hello world" shasum -a 1
<<< "hello world" shasum -a 1 | tr -d '\n -' | wc -c

12
22596363b3de40b06f981fb85d82312e8c0ed511  -
22596363b3de40b06f981fb85d82312e8c0ed511  -
40


In [6]:
!<<< "hello world, hello world, hello world, hello world" shasum

cf3400b0e78f01c31f1d5973100dffdaed0b4539  -


In [7]:
!<<< "hello world, hello world, hello wor1d, hello world" shasum

69a8a6c977609373ce8c7fcf29148f13f014b0f2  -


Reference: [Wikipedia on CHF]( https://en.wikipedia.org/wiki/Cryptographic_hash_function )


<br />
<br />
<br />
<br />
<br />

## Compression ( lossless )
- given input data, generate smaller output data ( usually )
- reversible: given output data, regenerate input data, exactly
- zip, gzip
- git uses zlib


In [8]:
%%bash
# compress
<<< "hello world, hello world, hello world, hello world" gzip > /tmp/qbf.compressed
# decompress
< /tmp/qbf.compressed gzip -dc

hello world, hello world, hello world, hello world


In [9]:
!< /tmp/qbf.compressed gzip -dc | shasum

cf3400b0e78f01c31f1d5973100dffdaed0b4539  -


Reference: [Wikipedia Compression]( https://en.wikipedia.org/wiki/Data_compression )

<br />
<br />
<br />
<br />
<br />

## Key-value store
- given a key, some value is returned
- DB: CRUD - create, remove, update, delete
- associative array ( awk ), hash ( perl, ruby ), dictionary (Python)
- implementations: Berkeley DB, redis, Dynamo, S3
- filesystem: no two files can have the same name

### Stiching the pieces together
- git uses sha-1 to generate a key, compresses the data, and then stores it on the filesystem
- result: 1-to-1 map of key to value

In [10]:
%%bash
# this is NOT what git does, but close

## Save to the K-V store, given some data
echo "hello world, hello world, hello world, hello world" > /tmp/example
key=$( < /tmp/example shasum | tr -d '\n -')
< /tmp/example gzip -c > /tmp/${key}

# verify
ls -lAF /tmp/example /tmp/"${key}"
file /tmp/example /tmp/"${key}"

## Retrieve from the K-V store, given some key
gzip -dc /tmp/"${key}"


-rw-r--r-- 1 jovyan users 37 Jun 10 01:12 /tmp/cf3400b0e78f01c31f1d5973100dffdaed0b4539
-rw-r--r-- 1 jovyan users 51 Jun 10 01:12 /tmp/example
/tmp/example:                                  ASCII text
/tmp/cf3400b0e78f01c31f1d5973100dffdaed0b4539: gzip compressed data, last modified: Thu Jun 10 01:12:21 2021, from Unix, original size modulo 2^32 51
hello world, hello world, hello world, hello world


<br />
<br />
<br />
<br />
<br />

## Graphs
- collection of nodes (vertices) and 0 or more edges that connect nodes
    - v3: solitary node
    - v1: single edge
    - v2: two edges
    - v4: two edges
    - v5: five edges, including a cycle
- undirected graph: the edge "connects" one node to another
    - (v1,v2), (v2,v1)
- directed graph: the edge "points" from one node to another
    - (v1,v2)
- path: finite or infinite sequence of edges that joins a sequence of vertices 
    - v1, v2, v5, v4
- directed acyclic graph ( DAG ): directed graphs with no directed cycles
- rooted in-tree: a DAG where all paths terminate to a single node ( the root )

![]( https://cdn.analyticsvidhya.com/wp-content/uploads/2018/03/graph1.png )


### Operations
- Adding a node
- Adding a directed edge
- Removing a directed edge
- Removing a node
